In [1]:
%load_ext autoreload
%autoreload 1
import sys
import numpy as np
import cv2
import torch

sys.path.append("/liujinxin/code/lhc/wy/openpi/src")
sys.path.append("/liujinxin/code/lhc/wy/openpi/packages/openpi-client/src")
sys.path.append("/liujinxin/code/lhc/wy/openpi")

from openpi.training import config as _config
from openpi.shared import download
from openpi.policies import policy_config as _policy_config

config = _config.get_config("pi0_bimanual_tool")
checkpoint_dir = download.maybe_download("/liujinxin/code/lhc/openpi/checkpoints/pi0_bimanual_tool/tool_filtered_678/15000")
policy = _policy_config.create_trained_policy(config, checkpoint_dir)

head_img = cv2.imread("/liujinxin/code/lhc/wy/openpi/rtc_debug/front_head_img.png")
left_img = cv2.imread("/liujinxin/code/lhc/wy/openpi/rtc_debug/left_hand_img.png")
right_img = cv2.imread("/liujinxin/code/lhc/wy/openpi/rtc_debug/right_hand_img.png")
state = np.load("/liujinxin/code/lhc/wy/openpi/rtc_debug/state.npy")
previous_actions = np.array([0])

instruction = "Put the round tape on the hook"
delay = 6
action_exec_s = 10

data = {
    "prompt": f"{instruction}",
    # "state": torch.from_numpy(state).to(dtype=torch.bfloat16),
    "state": torch.from_numpy(state).to(dtype=torch.float32),
    # "state": torch.from_numpy(state).to(dtype=torch.float32).zero_(),
    "front_head": torch.from_numpy(head_img),
    "left_hand": torch.from_numpy(left_img),
    "right_hand": torch.from_numpy(right_img),
}

/liujinxin/conda3/envs/openai_lhc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-08-21 09:28:36.094580: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-21 09:28:36.094717: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-21 09:28:36.273007: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-21 09:29:21.244308: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

In [2]:
%aimport openpi.models.pi0
%aimport openpi.policies.policy
%aimport openpi.policies.policy_config
config = _config.get_config("pi0_bimanual_tool")
checkpoint_dir = download.maybe_download("/liujinxin/code/lhc/openpi/checkpoints/pi0_bimanual_tool/tool_filtered_678/15000")
policy = _policy_config.create_trained_policy(config, checkpoint_dir)

In [3]:
policy._output_transform

CompositeTransform(transforms=[Unnormalize(norm_stats={'state': NormStats(mean=array([-1.39814147e-04, -4.86703975e-05, -5.00832160e-04,  3.80918173e-05,
        8.82984255e-04,  5.29210010e-05,  6.70640220e-05, -1.64848000e-01,
        7.68756643e-02,  3.27995978e-02, -2.96533555e-02, -1.73564687e-01,
        3.73103380e-01,  1.58386201e-01,  3.56420904e-01,  4.74421009e-02,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00]), std=array([1.00713922e-03, 8.14107829e-04, 3.46963265e-04, 8.37308122e-04,
       3.49176611e-04, 8.15438223e-04, 7.17441493e-04, 2.18300208e-01,
       6.53988644e-02, 2.06424966e-01, 2.26095900e-01, 1.20679080e-01,
       2.67902464e-01, 1.31154522e-01, 3.64534944e-01, 1.77853145e-02,
       0.00000000e+00, 0.000000

In [4]:
import time

previous_actions = np.array([0])
weight_vector = np.zeros(1600)

start = time.time()
action_rtc = policy.infer_rtc(data, previous_actions=previous_actions, weight_vector=weight_vector)
end = time.time()
print(f"Inference time: {end - start:.4f} seconds")

Inference time: 33.9191 seconds


In [5]:
start = time.time()
with torch.inference_mode():
    action = policy.infer(data)
end = time.time()
print(f"Inference time without RTC: {end - start:.4f} seconds")

result norm: 7.1267290115356445
Inference time without RTC: 10.1424 seconds


In [6]:
action["actions"].shape

(50, 16)

In [7]:
np.equal(action_rtc["actions"], action["actions"])[0, 0] == True

False

In [8]:
action_rtc["actions"].shape

(50, 16)

In [9]:
action["actions"]

array([[-1.02935029e-03, -5.27321714e-04, -6.71466115e-04,
        -7.08842654e-04,  1.02487201e-03, -4.15227742e-04,
        -1.00999615e-03,  5.13270380e-04,  1.61271956e-03,
        -1.71973249e-03, -3.07371705e-03,  1.14410689e-03,
        -9.55308012e-03, -7.15909002e-05,  6.17796565e-02,
         5.69912599e-02],
       [-1.02924767e-03, -5.27395625e-04, -6.71248926e-04,
        -7.13128488e-04,  1.02505536e-03, -4.16144026e-04,
        -1.01001795e-03,  2.04829045e-03,  2.31490774e-03,
        -2.51635744e-03, -2.22059002e-03,  1.74145512e-03,
        -9.31416893e-03, -3.64369554e-04,  5.83177626e-02,
         5.69907362e-02],
       [-1.02944023e-03, -5.27300250e-04, -6.71230299e-04,
        -7.10811661e-04,  1.02502396e-03, -4.15224307e-04,
        -1.00998468e-03,  1.92503683e-03,  3.29125627e-03,
        -3.74081508e-03, -2.47972931e-04,  2.29700002e-03,
        -8.03404369e-03, -6.05730141e-04,  5.79727735e-02,
         5.69747031e-02],
       [-1.02939202e-03, -5.27313152e

In [10]:

previous_actions_rtc = np.ones((50, 32))
import math
# move weight calculation outside of the inference code for compilation requirements
horizon = 50
delay = 10
action_exec_s = 10
# print("horizon:", horizon)
# print("delay:", delay)
weights = np.zeros(horizon)
for i in range(horizon):
    if i < delay:
        weights[i] = 1
    elif delay <= i < horizon - action_exec_s:
        c_i = (horizon - action_exec_s - i) / (horizon - action_exec_s - delay + 1)
        weights[i] = c_i * (math.exp(c_i) - 1) / (math.exp(1) - 1)
    else:
        weights[i] = 0
weight_vector = np.repeat(weights, 32)
print(weight_vector.shape)
# weights_expanded = torch.diag(weight_vector)
# weights_expanded = weights_expanded.numpy()

previous_actions_padded = np.pad(action["actions"], ((0, 0), (0, 16)), 'constant', constant_values=0)
start = time.time()
action_rtc = policy.infer_rtc(data, previous_actions=previous_actions_padded, weight_vector=weight_vector)
end = time.time()
print(f"Inference time: {end - start:.4f} seconds")

(1600,)


E0821 09:35:10.759844   28022 buffer_comparator.cc:156] Difference at 0: 0.503906, expected 6.1875
E0821 09:35:10.759935   28022 buffer_comparator.cc:156] Difference at 1: 0.554688, expected 6.59375
E0821 09:35:10.760060   28022 buffer_comparator.cc:156] Difference at 2: 0.546875, expected 6.53125
E0821 09:35:10.760068   28022 buffer_comparator.cc:156] Difference at 3: 0.523438, expected 6.78125
E0821 09:35:10.760074   28022 buffer_comparator.cc:156] Difference at 4: 0.511719, expected 6.6875
E0821 09:35:10.760080   28022 buffer_comparator.cc:156] Difference at 5: 0.660156, expected 7.03125
E0821 09:35:10.760086   28022 buffer_comparator.cc:156] Difference at 6: 0.660156, expected 6.71875
E0821 09:35:10.760092   28022 buffer_comparator.cc:156] Difference at 7: 0.632812, expected 6.4375
E0821 09:35:10.760098   28022 buffer_comparator.cc:156] Difference at 8: 0.597656, expected 6.65625
E0821 09:35:10.760104   28022 buffer_comparator.cc:156] Difference at 9: 0.667969, expected 6.65625
202

clipped bias norm: 0.13540802896022797
clipped bias norm: 0.06351350247859955
clipped bias norm: 0.042995255440473557
clipped bias norm: 0.0248564463108778
clipped bias norm: 0.023583920672535896
clipped bias norm: 0.02901471219956875
clipped bias norm: 0.04642451927065849
clipped bias norm: 0.10685074329376221
clipped bias norm: 0.2973184585571289
clipped bias norm: 0.7626121044158936
result_norm: 7.2220282554626465
Inference time: 20.8556 seconds


In [11]:
action_rtc["actions"]

array([[-0.00102906, -0.00052742, -0.00067087, -0.00070879,  0.00102454,
        -0.00041601, -0.00101   ,  0.00062138,  0.0011079 ,  0.00393835,
         0.00371621,  0.00056448, -0.01112909,  0.00167932,  0.063529  ,
         0.05694818],
       [-0.00102931, -0.00052727, -0.00067088, -0.00071253,  0.00102466,
        -0.00041578, -0.00100995,  0.00112442,  0.00184745,  0.00486684,
         0.00702598,  0.00098904, -0.00749964,  0.00185938,  0.06059031,
         0.05696274],
       [-0.00102933, -0.00052731, -0.00067086, -0.00070836,  0.00102485,
        -0.00041628, -0.00100996,  0.00018206,  0.00234817,  0.0050305 ,
         0.00933353,  0.00146517, -0.00589923,  0.00218216,  0.05923967,
         0.05694576],
       [-0.00102929, -0.00052722, -0.00067119, -0.00070841,  0.00102474,
        -0.00041604, -0.00100997,  0.00013207,  0.00294775,  0.00538227,
         0.01034392,  0.00169573, -0.0059551 ,  0.00173065,  0.05505532,
         0.05690487],
       [-0.00102933, -0.0005274 , -0

In [12]:
action_diff = action_rtc["actions"] - action["actions"]
action_diff[:6, 8:16]

array([[-5.04822350e-04,  5.65808725e-03,  6.78993004e-03,
        -5.79625659e-04, -1.57600741e-03,  1.75090833e-03,
         1.74933862e-03, -4.30766576e-05],
       [-4.67459502e-04,  7.38320166e-03,  9.24656683e-03,
        -7.52415458e-04,  1.81452617e-03,  2.22375112e-03,
         2.27254411e-03, -2.80010467e-05],
       [-9.43082698e-04,  8.77131505e-03,  9.58149811e-03,
        -8.31833514e-04,  2.13481747e-03,  2.78788567e-03,
         1.26689596e-03, -2.89404230e-05],
       [-5.99414481e-04,  9.42929966e-03,  8.76578022e-03,
        -9.95558409e-04,  1.74459520e-03,  2.34130981e-03,
        -2.30080921e-03, -4.32611400e-05],
       [-3.49854639e-04,  9.98793467e-03,  8.47218361e-03,
        -1.44588388e-03,  2.76753184e-03,  2.02658805e-03,
        -5.64688933e-03, -1.01066709e-04],
       [-3.96757959e-04,  9.97271095e-03,  9.16281565e-03,
        -1.04606153e-03,  1.69806285e-03,  1.78398135e-03,
        -5.09354731e-03, -1.74251118e-05]])

In [13]:
action["actions"][:6, 8:16]

array([[ 1.61271956e-03, -1.71973249e-03, -3.07371705e-03,
         1.14410689e-03, -9.55308012e-03, -7.15909002e-05,
         6.17796565e-02,  5.69912599e-02],
       [ 2.31490774e-03, -2.51635744e-03, -2.22059002e-03,
         1.74145512e-03, -9.31416893e-03, -3.64369554e-04,
         5.83177626e-02,  5.69907362e-02],
       [ 3.29125627e-03, -3.74081508e-03, -2.47972931e-04,
         2.29700002e-03, -8.03404369e-03, -6.05730141e-04,
         5.79727735e-02,  5.69747031e-02],
       [ 3.54716397e-03, -4.04702871e-03,  1.57813932e-03,
         2.69128958e-03, -7.69969360e-03, -6.10654851e-04,
         5.73561254e-02,  5.69481271e-02],
       [ 3.50877023e-03, -4.89442128e-03,  2.43300804e-03,
         3.48589220e-03, -8.29396949e-03, -4.41761353e-04,
         5.66732673e-02,  5.69811219e-02],
       [ 3.68631297e-03, -4.25785552e-03,  2.01427387e-03,
         3.07289241e-03, -6.18823106e-03, -1.17769337e-04,
         5.37194784e-02,  5.69349875e-02]])

In [14]:
action_diff[:6, 8:16] / action["actions"][:6, 8:16]

array([[-3.13025502e-01, -3.29009731e+00, -2.20902898e+00,
        -5.06618449e-01,  1.64973746e-01, -2.44571353e+01,
         2.83157712e-02, -7.55846732e-04],
       [-2.01934398e-01, -2.93408303e+00, -4.16401351e+00,
        -4.32061354e-01, -1.94813535e-01, -6.10301025e+00,
         3.89683007e-02, -4.91326285e-04],
       [-2.86541861e-01, -2.34476039e+00, -3.86392905e+01,
        -3.62139097e-01, -2.65721416e-01, -4.60252096e+00,
         2.18532923e-02, -5.07952150e-04],
       [-1.68984148e-01, -2.32993150e+00,  5.55450340e+00,
        -3.69918725e-01, -2.26579822e-01, -3.83409680e+00,
        -4.01144463e-02, -7.59658697e-04],
       [-9.97086205e-02, -2.04067735e+00,  3.48218480e+00,
        -4.14781583e-01, -3.33680013e-01, -4.58751776e+00,
        -9.96393821e-02, -1.77368759e-03],
       [-1.07630026e-01, -2.34219102e+00,  4.54894233e+00,
        -3.40415932e-01, -2.74401980e-01, -1.51480971e+01,
        -9.48175125e-02, -3.06052791e-04]])

In [15]:
action_frame_diff = action["actions"][1:, 8:16] - action["actions"][:-1, 8:16]

In [16]:
np.mean(action_frame_diff, axis=0)

array([ 7.82373801e-05,  2.12451595e-05,  2.44557893e-04,  8.23237804e-05,
        3.72611983e-05, -9.12572358e-07, -3.02443500e-04, -6.35410781e-07])

In [17]:
np.std(action_frame_diff, axis=0)

array([2.91507502e-04, 5.04595707e-04, 8.88340205e-04, 2.65451389e-04,
       7.62212007e-04, 2.97409448e-04, 1.03883278e-03, 3.68250019e-05])